# Iris Dataset - SOFENN Example

Demonstrate multi-class classification with SOFENN on classic Iris Dataset

In [1]:
import numpy as np
import random as rnd

from sklearn import datasets
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

import keras
import keras.backend as K

from keras.optimizers import SGD
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input

from sofenn.layers.DefuzzyLayer import DefuzzyLayer
from sofenn.layers import FuzzyLayer, NormalizedLayer, WeightedLayer, OutputLayer

import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [2]:
# custom loss function
def loss_function(y_true, y_pred):
    """
    Custom loss function

    E = exp{-sum[i=1,j; 1/2 * [pred(j) - test(j)]^2]}

    Parameters
    ==========
    y_true : np.array
        - true values
    y_pred : np.array
        - predicted values
    """
    return K.sum(1 / 2 * K.square(y_pred - y_true))

# read in iris dataset
iris = datasets.load_iris()
# create one-hot encoded vector for each class
Y=[]
for y in iris.target:
    tmp = np.zeros(3)
    tmp[y] = 1
    Y.append(tmp)

# split to train/test
X_train, X_test, y_train, y_test = train_test_split(iris.data, Y, test_size=0.1)

# set neurons
neurons = 3
K = neurons

# get random indexes for initial centers
indices = rnd.sample(range(len(X_train)), K)

# dataset info
samples, feats = X_train.shape

# create model layers
inputs = Input(name='Inputs', shape=(feats,))
fuzz = FuzzyLayer(neurons, input_dim=feats,
                  initializer_centers=
                  lambda x: np.transpose(np.array([X_train[i] for i in indices])))
#defuzz = DefuzzyLayer(5, input_dim=neurons)
norm = NormalizedLayer(neurons)
weights = WeightedLayer(neurons)
# raw = OutputLayer()
pred = Dense(3, activation='softmax')

# run through layers
phi = fuzz(inputs)
psi = norm(phi)
f = weights([inputs, psi])
# out = raw(f)
output = pred(f)

# define model
model = Model(inputs=inputs, outputs=output)

loss = 'mean_squared_error'
# loss = loss_function
optimizer = 'adam'
metrics = ['binary_accuracy']

model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

model.fit(np.array(X_train),
          np.array(y_train),
          epochs=50,
          verbose=1,
          batch_size=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
135/135 [==============================] - 3s 19ms/step - loss: 0.2118 - binary_accuracy: 0.6667
Epoch 2/50
135/135 [==============================] - 1s 4ms/step - loss: 0.1720 - binary_accuracy: 0.7185
Epoch 3/50
135/135 [==============================] - 1s 5ms/step - loss: 0.1342 - binary_accuracy: 0.8173
Epoch 4/50
135/135 [==============================] - 1s 4ms/step - loss: 0.1107 - binary_accuracy: 0.8173
Epoch 5/50
135/135 [==============================] - 1s 4ms/step - loss: 0.0932 - binary_accuracy: 0.8617
Epoch 6/50
135/135 [==============================] - 1s 5ms/step - loss: 0.0752 - binary_accuracy: 0.9284
Epoch 7/50
135/135 [==============================] - 1s 4ms/step - loss: 0.0564 - binary_accuracy: 0.9580
Epoch 8/50
135/135 [==============================] - 1s 4ms/step - loss: 0.0424 - binary_accuracy: 0.9605
Epoch 9/50
135/135 [===

KeyboardInterrupt: 

## Plot Results

In [ ]:
w = fuzz.get_weights()


plt.clf()
plt.title('Iris')
plt.ylabel('x[0]')
plt.xlabel('x[1]')
plt.scatter([a[0] for a in X_train], [a[1] for a in X_train], c=(0,0,0), alpha=0.5,s=1)
for i in range(0,K):
    ellipse = Ellipse((w[0][0][i], w[0][1][i]), w[1][0][i],w[1][1][i], color='r', fill=False)
    ax = plt.gca()
    ax.add_patch(ellipse)

plt.scatter(w[0][0], w[0][1], c=(1,0,0), alpha=0.8,s=15)
plt.show()

In [ ]:
import keras
from sofenn.layers import FuzzyLayer
from sofenn.layers.DefuzzyLayer import DefuzzyLayer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
import numpy as np

x_train = np.random.normal(5, 1, size=(100, 2))
y_train = np.random.normal(5, 15, size=(100, 4))

model = Sequential()
model.add(Dense(2, activation='sigmoid'))
model.add(FuzzyLayer(8))
model.add(Dense(8, activation='sigmoid'))
model.add(DefuzzyLayer(4))

model.compile(loss='logcosh',
              optimizer='rmsprop',
              metrics=['mae', 'acc'])

model.fit(x_train, y_train,
          epochs=10000,
          verbose=1,
          batch_size=100)

print(model.predict( np.array([[5, 5]])))
print(model.predict( np.array([[5, 15]])))
print(model.predict( np.array([[15, 5]])))
print(model.predict( np.array([[15, 15]])))